In [ ]:
from pathlib import Path
import json
import random
import numpy as np
#from CryptEnv import Environment 

In [ ]:
class Environment() :
    def __init__(self, exchange=0, path=None, dataSize=72, dataType='h', bonusData=12) :
        self.path = Path('data' if path==None else path)
        
        if isinstance(exchange, int) :
            exchanges = [ex for ex in self.path.glob('*') if ex.is_dir()]
            self.exchange = exchanges[exchange]
        elif isinstance(exchange, str) :
            self.exchange = self.path / exchange
            if not self.exchange.is_dir() : 
                raise ValueError("Exchange not found")

        self.dataSize = dataSize
        self.dataType = dataType
        self.bonusData = bonusData
        self.currencies = []
        self.load()
        self.cursor = 0
        
    def load(self) :
        datas = {}
        times = []
        for file in self.exchange.glob('*.' + self.dataType) :
            self.currencies.append(file.stem)
            data = json.loads(file.read_text())
            datas[file.stem] = { int(time):d for time, d in data.items() }
            times += datas[file.stem].keys()
            
        self.currencies.sort()
        times = list(set(times))
        times.sort()
        
        self.datas = [ {
                'time' : t,
                ** { c : 
                    ( datas[c][t] if t in datas[c] else {'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0 } )
                for c in self.currencies }
            } for i, t in enumerate(times) ]
            
    def getCurrencies(self) :
        return self.currencies
    
    def reset(self) :
        self.cursor = random.randrange(self.dataSize-1, len(self.datas) - self.bonusData)
    
    def readData(self) :
        self.cursor += 1
        if self.cursor + self.bonusData >= len(self.datas) : 
            return None
        
        return 
    
    

    
    def eval(self) :
        pass

In [ ]:
#import CryptEnv

c = Environment(exchange='Kraken')

In [ ]:
#print(c.path, c.exchange, c.datas['ADA-EUR'])
#print(len(c.datas), c.datas[49916])
random.seed(42)
c.reset()
c.cursor = 49900
print(c.cursor, c.dataSize, c.bonusData, len(c.datas), c.readData())